In [ ]:
# cpv No. 31(left) and 63(right): select ROI for cp in T1 FS space
# LV No. 4(left) and 43(right)

SUBJECTS_DIR='/home/binyin/nasdata/RJ_UI890_forAD/stacked_preproc/recon_fs711/'
subjDIR="/home/binyin/nasdata/RJ_UI890_forAD/niigz/"

Demo_RJ_bl_w_ab = pd.read_excel('/home/binyin/local4t/research/project/project_WMH_ADmarkers/WMH_marker_subROI.xlsx')

for i in range(len(Demo_RJ_bl_w_ab)):
    subj = Demo_RJ_bl_w_ab['ptid_viscode'][i]   
    if os.path.isfile(SUBJECTS_DIR+subj+'/mri/aseg_strucT1.nii.gz')==False:
        print(subj)
        
#subj='zzc19560602bl'
#aseg to T1_struct(T1_orig)
        !mri_label2vol --regheader $SUBJECTS_DIR/$subj/mri/aseg.mgz --seg $SUBJECTS_DIR/$subj/mri/aseg.mgz --temp $SUBJECTS_DIR/$subj/mri/rawavg.mgz --o $SUBJECTS_DIR/$subj/mri/aseg_strucT1.mgz
        !mri_convert $SUBJECTS_DIR/$subj/mri/aseg_strucT1.mgz $SUBJECTS_DIR/$subj/mri/aseg_strucT1.nii.gz

# make CP ROI in T1_brain space
        !fslmaths $SUBJECTS_DIR/$subj/mri/aseg_strucT1.nii.gz -uthr 31 -thr 31 -bin $SUBJECTS_DIR/$subj/mri/lh_cp.nii.gz
        !fslmaths $SUBJECTS_DIR/$subj/mri/aseg_strucT1.nii.gz -uthr 63 -thr 63 -bin $SUBJECTS_DIR/$subj/mri/rh_cp.nii.gz
        !fslmaths $SUBJECTS_DIR/$subj/mri/lh_cp.nii.gz -add $SUBJECTS_DIR/$subj/mri/rh_cp.nii.gz $SUBJECTS_DIR/$subj/mri/cp.nii.gz
        !flirt -in $SUBJECTS_DIR/$subj/mri/cp.nii.gz -ref $subjTAR/$subj/T1/T1_unbiased_brain.nii.gz -applyxfm -usesqform -out $SUBJECTS_DIR/$subj/mri/cp_T1brain.nii.gz   
        !fslmaths $SUBJECTS_DIR/$subj/mri/cp_T1brain.nii.gz -thr 0.5 -bin $SUBJECTS_DIR/$subj/mri/cp_T1brain.nii.gz

# make Lateral Ventricle ROI in T1_brain space
        !fslmaths $SUBJECTS_DIR/$subj/mri/aseg_strucT1.nii.gz -uthr 4 -thr 4 -bin $SUBJECTS_DIR/$subj/mri/lh_lv.nii.gz
        !fslmaths $SUBJECTS_DIR/$subj/mri/aseg_strucT1.nii.gz -uthr 43 -thr 43 -bin $SUBJECTS_DIR/$subj/mri/rh_lv.nii.gz
        !fslmaths $SUBJECTS_DIR/$subj/mri/lh_lv.nii.gz -add $SUBJECTS_DIR/$subj/mri/rh_lv.nii.gz $SUBJECTS_DIR/$subj/mri/lv.nii.gz
        !flirt -in $SUBJECTS_DIR/$subj/mri/lv.nii.gz -ref $subjTAR/$subj/T1/T1_unbiased_brain.nii.gz -applyxfm -usesqform -out $SUBJECTS_DIR/$subj/mri/lv_T1brain.nii.gz   
        !fslmaths $SUBJECTS_DIR/$subj/mri/lv_T1brain.nii.gz -thr 0.5 -bin $SUBJECTS_DIR/$subj/mri/lv_T1brain.nii.gz

#FA, MD, FWF, CBF in T1_brain space
        !flirt -in $subjTAR/$subj/dMRI/NODDI/AMICO/NODDI/fit_FWF.nii.gz -ref $subjTAR/$subj/T1/T1_unbiased_brain.nii.gz -o $subjTAR/$subj/dMRI/NODDI/AMICO/NODDI/fit_FWF_T1brain.nii.gz
        !flirt -in $subjTAR/$subj/dMRI/dtifit/dti_FA.nii.gz -ref $subjTAR/$subj/T1/T1_unbiased_brain.nii.gz -o $subjTAR/$subj/dMRI/dtifit/FA_T1brain.nii.gz
        !flirt -in $subjTAR/$subj/dMRI/dtifit/dti_MD.nii.gz -ref $subjTAR/$subj/T1/T1_unbiased_brain.nii.gz -o $subjTAR/$subj/dMRI/dtifit/MD_T1brain.nii.gz
        !flirt -in $subjTAR/$subj/asl_cbf/native_space/pvcorr/perfusion_calib.nii.gz -ref $subjTAR/$subj/T1/T1_unbiased_brain.nii.gz -o $subjTAR/$subj/asl_cbf/native_space/pvcorr/perfusion_calib_T1brain.nii.gz

In [ ]:
import numpy as np
import pandas as pd

Demo_RJ_bl_w_ab = pd.read_excel('/home/binyin/local4t/research/project/project_WMH_ADmarkers/WMH_marker_subROI_all.xlsx')

fwf_cp_list=[]
fa_cp_list=[]
md_cp_list=[]
cbf_cp_list=[]

for i in range(len(Demo_RJ_bl_w_ab)):
    subj = Demo_RJ_bl_w_ab['ptid_viscode'][i]   
    #print(i) 
    fwf_cp = !fslstats $subjTAR/$subj/dMRI/NODDI/AMICO/NODDI/fit_FWF_T1brain -k $SUBJECTS_DIR/$subj/mri/cp_T1brain -M
    fa_cp = !fslstats $subjTAR/$subj/dMRI/dtifit/FA_T1brain -k $SUBJECTS_DIR/$subj/mri/cp_T1brain -M
    md_cp = !fslstats $subjTAR/$subj/dMRI/dtifit/MD_T1brain -k $SUBJECTS_DIR/$subj/mri/cp_T1brain -M    
    cbf_cp = !fslstats $subjTAR/$subj/asl_cbf/native_space/pvcorr/perfusion_calib_T1brain -k $SUBJECTS_DIR/$subj/mri/cp_T1brain -M
    if len(fwf_cp[0])<11:
        fwf_cp_list.append(np.float32(fwf_cp)[0])
    else:
        fwf_cp_list.append('')
        
    if len(fa_cp[0])<11:    
        fa_cp_list.append(np.float32(fa_cp)[0])
    else:
        fa_cp_list.append('')
        
    if len(md_cp[0])<11:    
        md_cp_list.append(np.float32(md_cp)[0])
    else:
        md_cp_list.append('')
    
    if len(cbf_cp[0])<11:    
        cbf_cp_list.append(np.float32(cbf_cp)[0])
    else:
        cbf_cp_list.append('')
        